## Apress - Industrialized Machine Learning Examples

Andreas Francois Vermeulen
2019

### This is an example add-on to a book and needs to be accepted as part of that copyright.

## Chapter-011-22-Control-02

In [ ]:
# conda install -c conda-forge urllib3
# conda install -c conda-forge slycot
# conda install -c conda-forge control
# conda install -c plotly plotly

In [ ]:
#!pip install control

In [ ]:
#!pip install slycot

In [ ]:
#!pip install plotly

In [ ]:
from IPython.display import SVG
SVG('../../data/chapter 06/bicycle-control-design.svg')

# Controller Design

In [ ]:
SVG('../../data/chapter 06/bicycle-control-design2.svg')

The structure of the controller is shown in the following block diagram:

In [ ]:
import numpy as np
import control as cn
import plotly.plotly as pl
import plotly.graph_objs as gr

In [ ]:
g = 9.81  # m/s^2
m = 87.0  # kg
I = 3.28  # kg m^2
h = 1.0  # m
a = 0.5  # m
b = 1.0  # m
v = 5.0  # m/s

In [ ]:
num = -m * h * v / b * np.array([a, v])
den = np.array([(I + m * h**2), 0.0, -m * g * h])
theta_delta = cn.TransferFunction(num, den)
theta_delta

In [ ]:
theta_delta.pole()

In [ ]:
time = np.linspace(0.0, 5.0, num=1001)

In [ ]:
delta = np.deg2rad(2.0) * np.ones_like(time)

In [ ]:
time, theta, state = cn.forced_response(theta_delta, T=time, U=delta)

In [ ]:
def plot_siso_response(time, input, output, title='Time Response',
                       x_lab='Time [s]', x_lim=None,
                       input_y_lab='Input', input_y_lim=None,
                       output_y_lab='Output', output_y_lim=None,
                       subplots=True):
    """Plots a time history of the input and output of a SISO system."""
    
    xaxis = gr.XAxis(title=x_lab, range=x_lim)
    
    if subplots:
        yaxis = gr.YAxis(title=input_y_lab, range=input_y_lim, domain=[0.0, 0.49])
        yaxis2 = gr.YAxis(title=output_y_lab, range=output_y_lim, domain=[0.51, 1.0])
        layout = gr.Layout(title=title, xaxis=xaxis, yaxis=yaxis, yaxis2=yaxis2, showlegend=False)
        
        output_trace = gr.Scatter(name=output_y_lab, x=time, y=output, yaxis='y2')
    else:
        yaxis = gr.YAxis(range=output_y_lim)
        layout = gr.Layout(title=title, xaxis=xaxis, yaxis=yaxis)
        
        output_trace = gr.Scatter(name=output_y_lab, x=time, y=output)

    input_trace = gr.Scatter(name=input_y_lab, x=time, y=input)

    data = gr.Data([input_trace, output_trace])
    
    fig = gr.Figure(data=data, layout=layout)
    
    return fig

In [ ]:
pl.iplot(plot_siso_response(time, np.rad2deg(delta),np.rad2deg(theta), title='Step Response',
                            output_y_lab='Roll Angle [deg]', input_y_lab='Steer Angle [deg]'))

In [ ]:
def plot_root_locus(gains, roots):
    """Plots the root locus of the closed loop system given the provided gains."""
    
    real_vals = np.real(roots)
    imag_vals = np.imag(roots)
    
    xaxis = gr.XAxis(title='Re')
    yaxis = gr.YAxis(title='Im')
    layout = gr.Layout(title='Root Locus', showlegend=False,
                       xaxis=xaxis, yaxis=yaxis)
    
    # plots a blue "x" for the first roots
    open_loop_poles = gr.Scatter(x=real_vals[0, :],
                                 y=imag_vals[0, :],
                                 marker=gr.Marker(symbol='x', color='blue'),
                                 mode='markers')
    
    # plots a red "o" for the last roots
    last_poles = gr.Scatter(x=real_vals[-1, :],
                            y=imag_vals[-1, :],
                            marker=gr.Marker(symbol='o', color='red'),
                            mode='markers')
    data = []
    
    gain_text = ['k = {:1.2f}'.format(k) for k in gains]
    
    for r, i in zip(real_vals.T, imag_vals.T):
        data.append(gr.Scatter(x=r, y=i, text=gain_text,
                               marker=gr.Marker(color='black'), mode="markers"))
                
    data.append(open_loop_poles)
    data.append(last_poles)
    
    return gr.Figure(data=gr.Data(data), layout=layout)

In [ ]:
neg_feedback_roots, neg_feedback_gains = cn.root_locus(theta_delta, kvect=np.linspace(0.0, 10.0, num=500))

In [ ]:
pl.iplot(plot_root_locus(neg_feedback_gains, neg_feedback_roots))

In [ ]:
pos_feedback_roots, pos_feedback_gains = cn.root_locus(theta_delta, kvect=np.linspace(0.0, -20.0, num=500))
pl.iplot(plot_root_locus(pos_feedback_gains, pos_feedback_roots))

In [ ]:
def feedback(plant, controller):
    """Returns the closed loop system given the plant and controller of this form:
    
      +    -----   -----
    -->o-->| c |-->| p |--->
      -|   -----   ----- |
       -------------------
    
    """
    feedforward = controller * plant
    return (feedforward / (1 + feedforward)).minreal()

In [ ]:
k_theta = -2.5

In [ ]:
theta_thetac = feedback(theta_delta, k_theta)
#theta_thetac

In [ ]:
theta_thetac.pole()

In [ ]:
thetac = np.deg2rad(3.0) * np.ones_like(time)
time, theta, state = cn.forced_response(theta_thetac, T=time, U=thetac)

In [ ]:
pl.iplot(plot_siso_response(time, np.rad2deg(thetac), np.rad2deg(theta),
                            input_y_lab='Commanded Roll Angle [deg]',
                            output_y_lab='Roll Angle [deg]', subplots=False))

In [ ]:
thetae = thetac - theta
delta = k_theta * thetae

In [ ]:
pl.iplot(plot_siso_response(time, np.rad2deg(thetae), np.rad2deg(delta),
                           input_y_lab='Roll Error [deg]',
                           output_y_lab='Steer Angle [deg]'))

In [ ]:
#theta_thetac

In [ ]:
delta_theta = cn.TransferFunction(theta_delta.den, theta_delta.num)
#delta_theta

In [ ]:
psi_delta = cn.TransferFunction([v], [b, 0])
#psi_delta

In [ ]:
psi_thetac = (theta_thetac * delta_theta * psi_delta).minreal()
#psi_thetac

In [ ]:
psi_thetac.pole()

In [ ]:
psi_thetac.zero()

In [ ]:
time, psi, state = cn.forced_response(psi_thetac, T=time, U=thetac)

In [ ]:
pl.iplot(plot_siso_response(time, np.rad2deg(thetac), np.rad2deg(psi),
                            title="Step Response", output_y_lab='Heading Angle [deg]',
                            input_y_lab='Commanded Roll Angle [deg]'))

In [ ]:
roots, gains = cn.root_locus(psi_thetac, kvect=np.linspace(0.0, 3.0, num=1001))

In [ ]:
pl.iplot(plot_root_locus(gains, roots))

In [ ]:
k_psi = 0.25
psi_psic = feedback(psi_thetac, k_psi)
psi_psic.minreal()

In [ ]:
psi_psic.pole()

In [ ]:
psic = np.deg2rad(10.0) * np.ones_like(time)
time, psi, state = cn.forced_response(psi_psic, T=time, U=psic)

In [ ]:
pl.iplot(plot_siso_response(time, np.rad2deg(psic), np.rad2deg(psi),
                            input_y_lab="Commanded Heading [deg]",
                            output_y_lab="Heading [deg]",
                            subplots=False))

In [ ]:
psie = psic - psi
thetac = k_psi * psie
time, theta, state = cn.forced_response(theta_thetac, T=time, U=thetac)
thetae = thetac - theta
delta = k_theta * thetae

In [ ]:
xaxis = gr.XAxis(title='Time [s]')
    
yaxis = gr.YAxis(title='Steer [deg]', domain=[0.0, 0.32])
yaxis2 = gr.YAxis(title='Roll [deg]', domain=[0.33, 0.65])
yaxis3 = gr.YAxis(title='Heading [deg]', domain=[0.66, 1.0])

layout = gr.Layout(title='Commanded Heading Response', showlegend=False,
                   xaxis=xaxis, yaxis=yaxis, yaxis2=yaxis2, yaxis3=yaxis3)

steer_trace = gr.Scatter(x=time, y=np.rad2deg(delta))
roll_trace = gr.Scatter(x=time, y=np.rad2deg(theta), yaxis='y2')
heading_trace = gr.Scatter(x=time, y=np.rad2deg(psi), yaxis='y3')
commanded_heading_trace = gr.Scatter(x=time, y=np.rad2deg(psic), yaxis='y3')

data = gr.Data([steer_trace, roll_trace, heading_trace, commanded_heading_trace])
    
fig = gr.Figure(data=data, layout=layout)

pl.iplot(fig)

## Done

In [ ]:
import datetime
now = datetime.datetime.now()
print('Done!',str(now))